In [8]:
import os

api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

llm_model = 'gpt-3.5-turbo'


In [27]:
import json

def get_current_weather(location, unit= "celsious"):
    """ Get the weather of the given location """

    weather_info = {
        "location": location,
        "temperature": "25",
        "unit": unit,
        "forecast": ["sunny","windy"]
    }

    return json.dumps(weather_info)

In [19]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather for a location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state to get the weather for, e.g. San Fransisco, CA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["Celsius", "Fahrenheit"],
                    "description": "The temperature unit, either Celsius or Fahrenheit"
                }
            },
            "required": ["location"] 
        }
    }
]


In [20]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston, MA in celsious?"
    }
]

In [9]:
from openai import OpenAI

client = OpenAI(api_key=api_key)

In [21]:
response = client.chat.completions.create(
    model=llm_model,
    messages=messages,
    functions= functions
)

In [22]:
print(response)

ChatCompletion(id='chatcmpl-BFwAMcDZctbiGdYEeqC4VGpRwJnqc', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Boston, MA","unit":"Celsius"}', name='get_current_weather'), tool_calls=None))], created=1743138210, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=23, prompt_tokens=104, total_tokens=127, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [23]:
# Extract function call parameters
function_args = response.choices[0].message.function_call.arguments


print("Function Call Parameters:", function_args)

Function Call Parameters: {"location":"Boston, MA","unit":"Celsius"}


In [26]:
args = json.loads(response.choices[0].message.function_call.arguments)
print(args, type(args))

{'location': 'Boston, MA', 'unit': 'Celsius'} <class 'dict'>


In [30]:
get_current_weather(args)

'{"location": {"location": "Boston, MA", "unit": "Celsius"}, "temperature": "25", "unit": "celsious", "forecast": ["sunny", "windy"]}'

Function call , to force , auto, or none

In [31]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

response = client.chat.completions.create(
    model= llm_model,
    messages=messages,
    functions=functions,
    function_call="auto", # Decide whether to call weather function
)
print(response)

ChatCompletion(id='chatcmpl-BFwIHRucg42aBmD8pCyzHJH2tbXeS', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1743138701, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=11, prompt_tokens=92, total_tokens=103, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [33]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]

response = client.chat.completions.create(
    model= llm_model,
    messages=messages,
    functions=functions,
    function_call="none", # Setting up to not to call function
)
print(response)

ChatCompletion(id='chatcmpl-BFwL6enM9tjBg30WvKFsOlEFWjU46', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I can check the weather for you. Would you like the temperature in Celsius or Fahrenheit?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1743138876, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=19, prompt_tokens=98, total_tokens=117, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [35]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston, MA!",
    }
]
response = client.chat.completions.create(

    model=llm_model,
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}, #forcing to call the function
)
print(response)

ChatCompletion(id='chatcmpl-BFwT4tVhJxap7rcqtv7wdXpbunnUY', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather'), tool_calls=None))], created=1743139370, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=8, prompt_tokens=110, total_tokens=118, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [36]:
args = json.loads(response.choices[0].message.function_call.arguments)
print(args)

{'location': 'Boston, MA'}


In [37]:
observation = get_current_weather(args)

In [38]:
print(observation)

{"location": {"location": "Boston, MA"}, "temperature": "25", "unit": "celsious", "forecast": ["sunny", "windy"]}


In [41]:
messages.append(
    {
        "role": "function",
        "name": "get_current_weather",
        "content": observation
    }
)

response = client.chat.completions.create(
    model=llm_model,
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}
)

args = json.loads(response.choices[0].message.function_call.arguments)
print(args)

{'location': 'Boston, MA'}


In [42]:
print(get_current_weather(args))

{"location": {"location": "Boston, MA"}, "temperature": "25", "unit": "celsious", "forecast": ["sunny", "windy"]}
